### Load the model from Model Registry

In [1]:
import mlflow
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
name = 'monthly_sales'
stage = 'None'

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

<module 'mlflow.pyfunc' from '/Users/kanchanapadmanabhan/Library/CloudStorage/OneDrive-Personal/Personal-Course/Vector/Model Deployment/venv/lib/python3.8/site-packages/mlflow/pyfunc/__init__.py'>

In [11]:
model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")

In [12]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 991c3b49dd4c4fa589b1cbfe6e86ab49

In [25]:
model.metadata.get_input_schema()

['Store': long, 'Temperature': double, 'Fuel_Price': double, 'MarkDown1': double, 'MarkDown2': double, 'MarkDown3': double, 'MarkDown4': double, 'MarkDown5': double, 'CPI': double, 'Unemployment': double, 'IsHoliday': long, 'Size': long, 'Dept': long, 'month': long, 'monthly_sin_1': double, 'monthly_cos_1': double, 'monthly_sin_2': double, 'monthly_cos_2': double, 'monthly_sin_3': double, 'monthly_cos_3': double]

In [14]:
data = {'year_month':'2023-09',
        'Store': 1,
         'Temperature': 52.285,
         'Fuel_Price': 3.4595,
         'MarkDown1': 16931.265,
         'MarkDown2': 7281.18,
         'MarkDown3': 68.7925,
         'MarkDown4': 12626.244999999999,
         'MarkDown5': 6015.6725,
         'CPI': 220.37496355,
         'Unemployment': 7.348,
         'IsHoliday': 1,
         'Size': 151315,
         'Dept': 1,
     }

In [15]:
data['month'] = datetime.strptime(data['year_month'], '%Y-%m').month # Date.dt.to_period

In [16]:
def fourier_terms(value, period, num_terms):
    terms = []
    for i in range(1, num_terms + 1):
        terms.extend([np.sin(2 * np.pi * i * value / period),
                      np.cos(2 * np.pi * i * value / period)])
    term_names_sin =  ['monthly_sin_' + str(i) for i in range(1, num_terms+1)]
    term_names_cos =  ['monthly_cos_' + str(i) for i in range(1, num_terms+1)]
    term_names = [None]*(len(term_names_sin)+len(term_names_cos))
    term_names[::2] = term_names_sin
    term_names[1::2] = term_names_cos
    return dict(zip(term_names, terms))

In [17]:
# Define the period of each component (in days)
# For example, we can choose 365 days for yearly seasonality and 30 days for monthly seasonality.
period_monthly = 30

# Define the number of Fourier terms to use for each component
num_terms_monthly = 3


In [18]:
ft = fourier_terms(data['month'], 30, 3)

In [19]:
data.update(ft.items())

In [20]:
data

{'year_month': '2023-09',
 'Store': 1,
 'Temperature': 52.285,
 'Fuel_Price': 3.4595,
 'MarkDown1': 16931.265,
 'MarkDown2': 7281.18,
 'MarkDown3': 68.7925,
 'MarkDown4': 12626.244999999999,
 'MarkDown5': 6015.6725,
 'CPI': 220.37496355,
 'Unemployment': 7.348,
 'IsHoliday': 1,
 'Size': 151315,
 'Dept': 1,
 'month': 9,
 'monthly_sin_1': 0.9510565162951536,
 'monthly_cos_1': -0.30901699437494734,
 'monthly_sin_2': -0.587785252292473,
 'monthly_cos_2': -0.8090169943749475,
 'monthly_sin_3': -0.5877852522924734,
 'monthly_cos_3': 0.8090169943749473}

In [21]:
input_dict = {name:data[name] for name in model.metadata.get_input_schema().input_names()}

In [22]:
input_dict

{'Store': 1,
 'Temperature': 52.285,
 'Fuel_Price': 3.4595,
 'MarkDown1': 16931.265,
 'MarkDown2': 7281.18,
 'MarkDown3': 68.7925,
 'MarkDown4': 12626.244999999999,
 'MarkDown5': 6015.6725,
 'CPI': 220.37496355,
 'Unemployment': 7.348,
 'IsHoliday': 1,
 'Size': 151315,
 'Dept': 1,
 'month': 9,
 'monthly_sin_1': 0.9510565162951536,
 'monthly_cos_1': -0.30901699437494734,
 'monthly_sin_2': -0.587785252292473,
 'monthly_cos_2': -0.8090169943749475,
 'monthly_sin_3': -0.5877852522924734,
 'monthly_cos_3': 0.8090169943749473}

In [23]:
def get_prediction(data):
    result = model.predict(data)
    return result

In [24]:
get_prediction(input_dict)

array([81435.06], dtype=float32)